In [2]:
from dotenv import load_dotenv

import os

load_dotenv()

True

## Construção de Tools 

A construção de Tools com o objeto ``Structuredtool`` é importante devido a sua capacidade de ser compatível com as cadeias do ``Langchain``. Isso promove uma série de benefícios, como suporte para ``paralelização``, ``reutilização``, ``observabilidade``, ``monitoramento`` e outros.

Para a construção das ferramentas, podemos utilizar o código abaixo, o exemplo que vamos utilizar será reutilizado para construir uma ferramenta de agente que irá pesquisar tendências via ``Google Trends`` e retornará uma lista de conteúdos que estão em alta.

In [3]:
# Importar a biblioteca necessária
from pytrends.request import TrendReq
from langchain_core.tools import StructuredTool

# Inicializar um objeto TrendReq
pytrends = TrendReq(hl='pt-BR', tz=360)

def trends_per_country(country: str, head: int) -> str:
    """trends per country"""
    country = country.lower()
    daily_trending_searches = pytrends.trending_searches(pn=country)
    word_list = daily_trending_searches.head(head).values.T[0]
    return "\n".join(word_list)


async def atrends_per_country(country: str, head: int) -> str:
    """trends per country"""
    country = country.lower()
    daily_trending_searches = pytrends.trending_searches(pn=country)
    word_list = daily_trending_searches.head(head).values.T[0]
    return "\n".join(word_list)

def create_trends_tool():
    return StructuredTool.from_function(func=trends_per_country, coroutine=atrends_per_country)

trends = create_trends_tool()

print(trends.invoke({"country": "brazil", "head": 10}))
print("--------------------------------")
print(
    await trends.ainvoke({"country": "united_states", "head": 10})
)  # Uses use provided amultiply without additional overhead


Beatriz Haddad Maia
Cessar-fogo Gaza
AZUL4
Jogo joao Fonseca
Matheus Pereira
Cruzeiro
Palmeiras x Portuguesa
Barcelona x Betis
Volta Redonda x Fluminense
Boca Juniors x Juventude
--------------------------------
Red Dye 3
Ceasefire
Arsenal
Severance
Greater Los Angeles wildfires
TikTok banned
Daredevil: Born Again
Saif Ali Khan
Biden farewell address
Iowa State basketball


## Construção de Tools com o Serper

Aqui vamos construir outra ferramenta que irá pesquisar as tendências geradas pelo pytrends da etapa anterior. Abaixo você vai encontrar uma função que recebe uma lista de queries e retorna uma lista de notícias que foram retornadas pelo ``GoogleSerperAPIWrapper`` (Pesquise sobre o Serper para entender melhor).

Por fim, vamos acompanhar o código que será utilizado para construir a ferramenta.

In [4]:
from typing import List
import os

from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from tqdm import tqdm

def get_serper_results(queries: List[str],
                       k: int=5,
                       type_content: str='news',
                       hl: str='pt',
                       gl: str='br') -> List[dict]:
    
    """
    Get the multiple search results from Google Serper API
    
    Args:
    - queries: List of queries to be searched
    - k: Number of results to be returned
    - type_content: Type of content to be searched
    - hl: Language to be searched
    - gl: Country to be searched
    
    Returns:
    - List of results from the search
    
    """

    search = GoogleSerperAPIWrapper(gl=gl,
                                    hl=hl,
                                    k=k, 
                                    type=type_content) # type: Literal['news', 'search', 'places', 'images'] = 'search'
    
    results = [search.results(query) for query in queries]
    return results

def get_serper_with_scrapping(queries: List[str],
                              k: int=5,
                              type_content: str='news',
                              hl: str='pt',
                              gl: str='br') -> List[dict]:
    
    """
    Get the multiple search results from Google Serper API and scrap the content
    
    Args:
    - queries: List of queries to be searched
    - k: Number of results to be returned
    - type_content: Type of content to be searched
    - hl: Language to be searched
    - gl: Country to be searched
    
    Returns:
    - List of results from the search
    
    """
    
    results = get_serper_results(queries, 
                                 k, 
                                 type_content, 
                                 hl, 
                                 gl)
    
    dict_results_news = {}
    for r in tqdm(results):
        q = r['searchParameters']['q']
        news = r['news']
        
        for i in range(len(news)):
            n = news[i]
            link = n['link']
            loader = WebBaseLoader(web_paths=[link])
            docs = loader.load()
            news[i]['content'] = "\n".join([x.page_content for x in docs])
        
        dict_results_news[q] = news
        
    return dict_results_news

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
def to_list(text):
    return text.split("\n")

trends_text = trends.invoke({"country": "brazil", 
                             "head": 3})
trends_list = to_list(trends_text)
trends_list

['Beatriz Haddad Maia', 'Cessar-fogo Gaza', 'AZUL4']

In [6]:
contents_news = get_serper_with_scrapping(trends_list, k=5)

100%|██████████| 3/3 [00:10<00:00,  3.48s/it]


In [7]:
contents_news

{'Beatriz Haddad Maia': [{'title': 'João Fonseca e Bia Haddad têm destinos diferentes no Australian Open',
   'link': 'https://www.terra.com.br/esportes/tenis/joao-fonseca-e-bia-haddad-tem-destinos-diferentes-no-australian-open,e5ba253fafc4b493303b219c76c11dc3vgg2a7ma.html',
   'snippet': 'A madrugada no Australian Open 2025 trouxe emoções contrastantes para os torcedores brasileiros. Beatriz Haddad Maia e João Fonseca,...',
   'date': 'há 2 horas',
   'source': 'Terra',
   'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQnmp4QZ-DzxVQoCrYCkmuSIK8wPAX-bQHp9O4U70Jyz1_e21TBQW8vHQ4&usqp=CAI&s',
   'position': 1,
   'content': "\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJoão Fonseca e Bia Haddad têm destinos diferentes no Australian Open\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nScript = https://s1.trrsf.com/update-1736967909/fe/zaz-ui-t360/_js/transition.min.js\nScript = https://pagead2.googlesyndication.com/pagead/

## Conclusão:

Em primeiro lugar, é possível observar a utilidade de construir ferramentas (Tools). Elas são capazes de facilitar a reutilização de código, a paralelização de tarefas, a observabilidade e o monitoramento de tarefas. Por fim, vamos utilizar essas ferramentas para construir um agente que irá pesquisar tendências via ``Google Trends`` e retornar uma lista de conteúdos que estão em alta, a partir de pesquisas na web.